In [4]:
#Bibliotecas
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from numpy.linalg import inv
from numpy.linalg import pinv
import sys


In [1]:
# Esta função realiza a calibração de um sistema de câmera usando o método DLT (método das diretas).
# Ela recebe como parâmetro os pontos 3D (cp3d) e 2D (cp2d) e retorna a matriz DLT.
# Essa matriz contém os parâmetros que descrevem a relação entre os dois sistemas de coordenadas.

def calibração_DLT(pontos_3D, pontos_2D):

    pontos_3D = np.asarray(pontos_3D)
    pontos_2D = np.asarray(pontos_2D)

    qtdade_pontos = np.size(pontos_3D[:, 0], 0)
    matriz_M = np.zeros([qtdade_pontos * 2, 11])
    vetor_N = np.zeros([qtdade_pontos * 2, 1])

    for i in range(qtdade_pontos):
        matriz_M[i*2,:] = [pontos_3D[i,0], pontos_3D[i,1], pontos_3D[i,2], 1, 0, 0, 0, 0, -pontos_2D[i, 0] * pontos_3D[i, 0], -pontos_2D[i, 0] * pontos_3D[i, 1], -pontos_2D[i, 0] * pontos_3D[i, 2]]
        matriz_M[i*2+1,:] = [0 , 0, 0, 0, pontos_3D[i,0], pontos_3D[i,1], pontos_3D[i,2],1, -pontos_2D[i,1] * pontos_3D[i,0],-pontos_2D[i,1] * pontos_3D[i,1], -pontos_2D[i,1] * pontos_3D[i,2]]
        vetor_N[[i*2,i*2+1],0] = pontos_2D[i, :]

    matriz_M_transposta = matriz_M.T
    matriz_M1 = inv(matriz_M_transposta.dot(matriz_M))
    matriz_M_vetor_N = matriz_M_transposta.dot(vetor_N)

    DLT = (matriz_M1).dot(matriz_M_vetor_N).T

    return DLT


#Função de reconstrução em 3 Dimensões
def reconstrucao_3d(DLTs, cc2ds):
    DLTs = np.asarray(DLTs)
    cc2ds = np.asarray(cc2ds)
    
    m = len(DLTs)  #Quantidade de equações 
    M = np.zeros([2 * m, 3]) #Matriz com a quantidade de equações (2m) e 3 colunas
    N = np.zeros([2 * m, 1]) #Matriz com a quantidade de equações (2m) e 1 coluna

    #Preenchendo as matrizes
    for i in range(m):
        M[i*2,:] = [DLTs[i,0]-DLTs[i,8]*cc2ds[i,0], DLTs[i,1]-DLTs[i,9]*cc2ds[i,0], DLTs[i,2]-DLTs[i,10]
        *cc2ds[i,0]] #Primeira linha da matriz M
        M[i*2+1,:] = [DLTs[i,4]-DLTs[i,8]*cc2ds[i,1],DLTs[i,5]-DLTs[i,9]*cc2ds[i,1],DLTs[i,6]-DLTs[i,10]
        *cc2ds[i,1]] #Segunda linha da matriz M
        Np1 = cc2ds[i,:].T
        Np2 = [DLTs[i,3],DLTs[i,7]]
        N[[i*2,i*2+1],0] = Np1 - Np2 #Atribuindo os valores à matriz N

    cc3d = inv(M.T.dot(M)).dot((M.T.dot(N)))  #Calcula as coordenadas 3D
    
    return cc3d

# %% Executar na IDE Python
def rec3d_ide(c1=None, c2=None, ref=None):
    # Para teste na IDE
    if c1 is None:
        dfcp2d_c1 = pd.read_csv('cp2d_c1.txt', delimiter=' ',header=None) #Lê o arquivo de coordenadas 2D da câmera 1
        dfcp2d_c2 = pd.read_csv('cp2d_c2.txt', delimiter=' ',header=None) #Lê o arquivo de coordenadas 2D da câmera 2
        dfcp3d = pd.read_csv('cp3d.txt', delimiter=' ',header=None) #Lê o arquivo de coordenadas 3D
    else:
        dfcp2d_c1 = c1
        dfcp2d_c2 = c2
        dfcp3d = ref
            
    cp2dc1 = np.asarray(dfcp2d_c1) #Matriz com as coordenadas 2D da câmera 1
    cp2dc2 = np.asarray(dfcp2d_c2) #Matriz com as coordenadas 2D da câmera 2
    cp3d = np.asarray(dfcp3d) #Matriz com as coordenadas 3D
    
    dltc1 = dlt_calib(cp3d, cp2dc1) #Calcula os parâmetros de calibragem da câmera 1
    dltc2 = dlt_calib(cp3d, cp2dc2) #Calcula os parâmetros de calibragem da câmera 2
    
    DLTs = np.append(dltc1, dltc2, axis=0) #Concatena os parâmetros de calibragem das duas câmeras
    
    cc3d = np.zeros([len(cp2dc1), 3]) #Matriz que armazenará as coordenadas 3D
    for i in range(len(cp2dc1)):
        cc2ds = np.append([cp2dc1[i, :]], [cp2dc2[i, :]], axis=0) #Concatena as coordenadas 2D das duas câmeras
        cc3d[i, :] = r3d(DLTs, cc2ds).T #Atribui as coordenadas 3D à matriz de saída
    
    # print(cc3d)
    
    # np.savetxt(input('Nome do arquivo: ')+'.3d', cc3d, fmt='%.10f')
    return cc3d #Retorna a matriz com as coordenadas 3D


def cart2sph(x,y,z):
    azimuth = np.arctan2(y,x) #Calcula o ângulo de azimute
    elevation = np.arctan2(z,np.sqrt(x**2 + y**2)) #Calcula o ângulo de elevação
    r = np.sqrt(x**2 + y**2 + z**2) #Calcula a distância do ponto
    return azimuth, elevation, r



In [ ]:

# Função Main para a execução do código: 
if __name__ == '__main__':
    print('\n')
    print(58*'#')
    print('KVELBALL.PY'.center(58))
    print('Velocidade da bola'.center(58))
    print('Prof. PAULO R. P. SANTIAGO'.center(58))
    print('LaBioCoM-EEFERP-USP'.center(58))
    print('paulosantiago@usp.br'.center(58))
    print('Created on 12/04/2020 - Update on 01/06/2020'.center(58))
    print('Como rodar:\n python kvelball.py c1.txt c2.txt c1cal.txt c2cal.txt calibrador_ref.txt nome_que_deseja_salvar'.center(58))
    print(58*'#')
    print('\n')

    # Definindo os parâmetros das cameras do experimento
    resolutionx = int(720/2) # Arrumar a translação X do sist. ref do kinovea 0.9.4
    resolutiony = int(220/2) # Arrumar a translação Y do sist. ref do kinovea 0.9.4
    freq = 120 # Frequência de amostragem
    
    # Carregando os arquivos das cameras do Kinovea
    # Camera 1
    bola1 = pd.read_csv(str(sys.argv[1]), sep='\s+', header=None, decimal='.')
    bola1[1] = bola1[1] - -resolutionx
    bola1[2] = -1 * (bola1[2] - resolutiony) 
    bola1 = np.asarray(bola1[[1,2]])
    bola1b = bola1
    
    # Camera 2
    bola2 = pd.read_csv(str(sys.argv[2]), sep='\s+', header=None, decimal='.')
    bola2[1] = bola2[1] - -resolutionx
    bola2[2] = -1 * (bola2[2] - resolutiony) 
    bola2 = np.asarray(bola2[[1,2]])
    bola2b = bola2
    
    idx = np.asarray(list(range(len(bola1b))))
    diffball = abs(np.diff(bola1b[:,0])) > 5
    diffball = np.insert(diffball, 0, False)
    phitball = idx[diffball][0]
    idxbefore = idx[0:phitball-2]
    idxafter = idx[phitball+1::]
    idxcimpact = idx[phitball-2:phitball+1]

    print(f'Frame of impact = {phitball}')
    print(f'Critical impact frames  = {idxcimpact}')

    plt.close('all')
    plt.subplot(2,1,1)
    plt.grid(True)
    plt.plot(bola1[:,0],bola1[:,1],'o')
    plt.xlabel('CAM 1 - Coordinate X')
    plt.ylabel('CAM 1 - Coordinate Y')
    resx = 2 * resolutionx
    resy = 2 * resolutiony
    plt.title(f'Pixels coordinates (resolution = {resx} X {resy})')
    
    plt.subplot(2,1,2)
    plt.plot(bola2[:,0],bola2[:,1],'o')
    plt.xlabel('CAM 2 - Coordinate X')
    plt.ylabel('CAM 2 - Coordinate Y')
    plt.grid(True)
    
    # Carregar arquivos de calibracao 
    datcal_c1 = np.asarray(pd.read_csv(str(sys.argv[3]), sep='\s+', header=None))
    datcal_c1[:, 0] = datcal_c1[:, 0] - -resolutionx
    datcal_c1[:, 1] = -1 * (datcal_c1[:, 1] - resolutiony) 
    
    datcal_c2 = np.asarray(pd.read_csv(str(sys.argv[4]), sep='\s+', header=None))
    datcal_c2[:, 0] = datcal_c2[:, 0] - -resolutionx
    datcal_c2[:, 1] = -1 * (datcal_c2[:, 1] - resolutiony) 

    
    ref = np.asarray(pd.read_csv(sys.argv[5], sep='\s+', header=None))
    ref = ref[:,1:]
    
    
    dltc1 = dlt_calib(ref, datcal_c1)
    dltc2 = dlt_calib(ref, datcal_c2)
    dlts = np.append(dltc1, dltc2, axis=0)
    
    cc3d = np.zeros([len(bola1), 3])
    
    for i in range(len(bola1)):
        cc2ds = np.append([bola1[i, :]], [bola2[i, :]], axis=0)
        cc3d[i, :] = r3d(dlts, cc2ds).T
    
    cc3df = cc3d[idxafter,:]
    coefsx = np.polyfit(idxafter, cc3df[:,0], 1)
    coefsy = np.polyfit(idxafter, cc3df[:,1], 1)
    coefsz = np.polyfit(idxafter, cc3df[:,2], 2)
    
    cc3df[:,0] = coefsx[0] * idxafter + coefsx[1]
    cc3df[:,1] = coefsy[0] * idxafter + coefsy[1]
    cc3df[:,2] = coefsz[0] * idxafter**2 + coefsz[1] * idxafter + coefsz[2]
    
    vels = (np.sqrt(np.sum((np.diff(cc3df, axis=0)**2), axis=1))) / (1/freq) * 3.6
    print(f'Speeds = {vels}')

    vsaida = cc3df[-1:,:] - cc3d[0,:]

    azimuth, elevation, r = cart2sph(vsaida[0][0], vsaida[0][1], vsaida[0][2])
    pi = np.pi
    azi = azimuth * 180/pi
    elev = elevation * 180/pi
    
    print(f'Angles: azimuth = {azi}; elevation = {elev}')
   
    fig2 = plt.figure()
    ax2 = fig2.add_subplot(111, projection='3d') 
    ax2.plot3D(cc3d[:,0], cc3d[:,1], cc3d[:,2], 'ro', markersize=10)
    ax2.plot3D(ref[:,0], ref[:,1], ref[:,2], 'b.')
    ax2.plot3D(cc3df[:,0], cc3df[:,1], cc3df[:,2], 'k-o')
    ax2.plot3D([cc3d[0,0],cc3d[0,0]], [cc3d[0,1],cc3d[0,1]], [cc3d[0,2],cc3d[0,2]], 'g.', markersize=10)
    
    ax2.set_zlabel('Z [m]')
    ax2.set_ylabel('Y [m]')
    ax2.set_xlabel('X [m]')
    
    # print(cc3df)
    distvet = np.sqrt(np.sum((cc3df[-1,:] - cc3df[0,:])**2))
    
    velmed = distvet / (len(cc3df) * (1/freq)) * 3.6
    plt.title(f'Speed (Max = {np.round(max(vels),2)} km/h ; Mean = {np.round(velmed)}); Angles (azi = {np.round(azi,1)}, elev. = {np.round(elev,1)})')
    plt.show()

    # import pdb; pdb.set_trace() 
    resultado = list(np.append(vels, [azi, elev, velmed]))
    np.savetxt(str(sys.argv[6])+'_result.txt', resultado, fmt='%.10f')
    print(f'Mean Speed = {velmed}')
    print('\n')
    
    # with open(str(sys.argv[6])+'_res.txt', 'w') as output:
    #     output.write(str(resultado))
   
    np.savetxt(str(sys.argv[6])+'.3d', cc3d, fmt='%.10f')
    np.savetxt(str(sys.argv[6])+'_filt.3d', cc3df, fmt='%.10f')

    # np.savetxt(str(sys.argv[6])+'.txt', resultado, fmt='%.10f')

    np.savetxt(str(sys.argv[6])+'.3d', cc3d, fmt='%.10f')